In [1]:
import sys, os

ppath = lambda x: os.path.dirname(os.path.abspath(x))
file_name = os.getcwd()
sys.path.append(ppath(ppath(file_name)))

from recorder.recorder import *

In [2]:
import gym

from ppo import *
from torch.distributions import Categorical

In [3]:
def main():
    recorder = Recorder(gym.make('CartPole-v1'), False)
    env = recorder.wrapped_env()
    model = PPO()
    score = 0.0
    print_interval = 20

    for n_epi in range(10000):
        s = env.reset()
        done = False
        while not done:
            for t in range(T_horizon):
                prob = model.pi(torch.from_numpy(s).float())
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)
                
                # r은 크기가 너무 커서 1/100을 했다. 그래야 학습이 잘 됨.
                # prob[a].item()은 a를 할 확률을 말함
                model.put_data((s, a, r/100.0, s_prime, prob[a].item(), done))
                s = s_prime

                score += r
                if done:
                    break

            model.train_net()
        
        if score / print_interval > 499:
            recorder.update([n_epi + 1])
            if (len(recorder.recorded_epi()) > 5):
                env.reset()
                break

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode: {}, avg score: {:.1f}".format(n_epi, score/print_interval))
            score = 0.0

    env.close()

In [4]:
if __name__ == '__main__':
    main()

# of episode :20, avg score : 37.1
# of episode :40, avg score : 45.5
# of episode :60, avg score : 60.6
# of episode :80, avg score : 74.5
# of episode :100, avg score : 35.0
# of episode :120, avg score : 80.5
# of episode :140, avg score : 73.8
# of episode :160, avg score : 129.2
# of episode :180, avg score : 94.8
# of episode :200, avg score : 163.7
# of episode :220, avg score : 127.8
# of episode :240, avg score : 250.9
# of episode :260, avg score : 247.9
# of episode :280, avg score : 217.1
# of episode :300, avg score : 276.6
# of episode :320, avg score : 288.4
# of episode :340, avg score : 326.6
# of episode :360, avg score : 245.8
# of episode :380, avg score : 142.8
# of episode :400, avg score : 205.7
# of episode :420, avg score : 274.6
# of episode :440, avg score : 361.6
# of episode :460, avg score : 451.1
# of episode :480, avg score : 433.4
# of episode :500, avg score : 397.1
# of episode :520, avg score : 316.7
# of episode :540, avg score : 216.7
# of episode 